# Logistic Regression Code Along

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('titanic').getOrCreate()

In [3]:
spark

In [4]:
df = spark.read.csv('./data/titanic.csv',inferSchema = True , header = True)

In [5]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [6]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [7]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

## Categorical Columns

In [8]:
# Seleccionamos columnas que vamos a usar 
my_cols = df.select(['Survived','Pclass','Sex','Age','SibSp','Parch', 'Fare','Embarked'])

In [36]:
# Missing data
my_final_data = my_cols.na.drop()

In [18]:
# Categorical columns
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [39]:
# Crear categorías númericas para categorías con string

gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
# A B C
# 0 1 2
# One hot encode
# example A
# [1, 0, 0]
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol= 'SexVec')

In [25]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

Exception ignored in: <function JavaWrapper.__del__ at 0x7ffa002f6dc0>
Traceback (most recent call last):
  File "/home/patricia/miniconda3/envs/spark-course/lib/python3.8/site-packages/pyspark/ml/wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'OneHotEncoder' object has no attribute '_java_obj'


In [26]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [28]:
from pyspark.ml.classification import LogisticRegression

In [30]:
from pyspark.ml import Pipeline

## Pipelines 

In [42]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [43]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [44]:
train_data,test_data = my_final_data.randomSplit([0.7,0.3])

In [45]:
fit_model = pipeline.fit(train_data)

In [46]:
results = fit_model.transform(test_data)

In [47]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [48]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction',labelCol = 'Survived')

In [49]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [50]:
AUC = my_eval.evaluate(results)

In [52]:
AUC

0.7980147323032698